In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import numba as nb
from numbalsoda import lsoda, address_as_void_pointer

from src.lr import gernerate_full_lr_with_oxalate
from src.loss import cr_log, args_dtype_cr, create_jit_cr
from src.chemkinetics import ChemKinetics, find_sparse_model
from src.load_data import generate_d
from src.tools import get_conservation, generate_chem_formula, chem_equation_with_oxalate
from src.report import report

In [2]:
# set the number of threads to be used by numba
#print(nb.config.NUMBA_DEFAULT_NUM_THREADS)
nb.config.NUMBA_NUM_THREADS = 8
print(nb.config.NUMBA_NUM_THREADS)

# create the function (rate equation in this study) to be called by numbalsoda
cr_cfunc = create_jit_cr(cr_log, args_dtype_cr)
funcptr_cr = cr_cfunc.address

8


In [3]:
conservation = [[1, 1, 1, 0, 0, 1, 1, 0,], # number of Mn atoms
                [0, 0, 0, 2, 1, 0, 0, 2,], # number of C atoms
                [7, 3, 2,-2, 0, 6, 4,-1,], # charge state
               ] # Mn(7+), Mn(3+), Mn(2+), C2O4(2-), CO2, Mn(6+), Mn(4+), C2O4(-)

chemformula = generate_chem_formula(conservation)
l,r = gernerate_full_lr_with_oxalate(conservation)
print(chem_equation_with_oxalate(l, r, k=None, chemformula=chemformula, conservation=conservation))

( 1) Mn+7 + Mn+3     -> Mn+6 + Mn+4     k = 0.00
( 2) Mn+7 + Mn+2     -> Mn+3 + Mn+6     k = 0.00
( 3) Mn+7 + C2O4-2   -> Mn+6 + C2O4-1   k = 0.00
( 4) 2 Mn+3          -> Mn+2 + Mn+4     k = 0.00
( 5) Mn+3 + C2O4-2   -> Mn+2 + C2O4-1   k = 0.00
( 6) Mn+3 + Mn+6     -> Mn+7 + Mn+2     k = 0.00
( 7) Mn+3 + C2O4-1   -> C2O4-2 + Mn+4   k = 0.00
( 8) Mn+2 + Mn+6     -> 2 Mn+4          k = 0.00
( 9) Mn+2 + Mn+4     -> 2 Mn+3          k = 0.00
(10) Mn+2 + C2O4-1   -> Mn+3 + C2O4-2   k = 0.00
(11) C2O4-2 + Mn+4   -> Mn+3 + C2O4-1   k = 0.00
(12) Mn+6 + Mn+4     -> Mn+7 + Mn+3     k = 0.00
(13) Mn+6 + C2O4-1   -> Mn+7 + C2O4-2   k = 0.00
(14) 2 Mn+4          -> Mn+2 + Mn+6     k = 0.00
(15) Mn+7 + C2O4-1   -> 2 CO2 + Mn+6    k = 0.00
(16) Mn+3 + C2O4-1   -> Mn+2 + 2 CO2    k = 0.00
(17) C2O4-2 + Mn+6   -> 2 CO2 + Mn+4    k = 0.00
(18) C2O4-2 + Mn+4   -> Mn+2 + 2 CO2    k = 0.00
(19) Mn+4 + C2O4-1   -> Mn+3 + 2 CO2    k = 0.00
(20) 2 C2O4-1        -> C2O4-2 + 2 CO2  k = 0.00



In [4]:
# find full models (lam = 0) and models for any given lam
savefolder = "example/init"
savename = "0000"
lam = 0.
target = 0. 
# Once the MRSE value decreases to less than the target value, the tentative best result is injected as an external solution to CMA-ES.
# This does not work when target value = 0.

d, d_test = generate_d()
ck = ChemKinetics(funcptr_cr, d, d_test=d_test, l=l, r=r, conservation=conservation, chemformula=chemformula)
ck.optimize(lam = lam, target=target)
ck.save(savefolder, savename)

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=254621, Sun Apr 20 17:16:32 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 9.719892259896626e-01 1.0e+00 9.32e-01  9e-01  9e-01 0:01.3
    2     24 1.040073863777357e+00 1.1e+00 8.99e-01  9e-01  9e-01 0:01.3
    3     36 8.740413655926046e-01 1.1e+00 8.86e-01  9e-01  9e-01 0:01.3
  100   1200 1.530423916770432e-01 3.1e+00 4.75e-01  3e-01  6e-01 0:03.5
  200   2400 2.015143574618226e-02 6.8e+00 1.06e-01  6e-02  1e-01 0:06.0
  300   3600 1.627627756091865e-02 2.0e+01 3.84e-02  1e-02  5e-02 0:08.4
  400   4800 1.587954825320617e-02 5.6e+01 1.78e-02  4e-03  3e-02 0:10.8
  500   6000 1.186966455417538e-02 1.1e+02 1.03e-01  1e-02  2e-01 0:13.2
  600   7200 7.834805844990101e-03 1.0e+02 6.28e-02  8e-03  1e-01 0:15.7
  700   8400 5.412636396130958e-03 1.2e+02 2.01e-02  2e-03  3e-02 0:18.1
  800   9600 5.300201728671242e-03 1.3e+02 9.74e-03  5e-04  1e-02 0:20.6
  900  10800 4.903402525903228e-0

lsoda -- warning..internal t = 0.660373 and h_ = 4.55617e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.660373 and h_ = 4.55617e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.660373 and h_ = 3.08524e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.660373 and h_ = 3.08524e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.660373 and h_ = 2.08919e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.660373 and h_ = 2.08919e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsod

  200  16051 1.299120137178227e-02 6.7e+00 5.65e-02  2e-02  7e-02 0:05.7
  300  17451 1.000706485608763e-02 3.0e+01 4.96e-02  5e-03  7e-02 0:08.8
  400  18851 5.834959387993459e-03 6.5e+01 7.15e-02  5e-03  1e-01 0:11.9
  500  20251 5.239700816623037e-03 1.3e+02 4.89e-02  2e-03  7e-02 0:15.0
  600  21651 4.865482039516533e-03 1.6e+02 2.42e-02  8e-04  3e-02 0:17.9
  700  23051 4.838709837308795e-03 2.2e+02 4.34e-03  1e-04  6e-03 0:21.0
  800  24451 4.836489533784637e-03 2.5e+02 1.44e-03  3e-05  2e-03 0:24.0
  860  25291 4.836005609447783e-03 2.7e+02 5.46e-04  1e-05  7e-04 0:25.8
termination: {'tolfun': 1e-06}
{'compared': 12040,
 'evals': 25188,
 'evalsall': 25291,
 'f': 0.004835971081751055,
 'last': <cma.utilities.utils.BlancClass object at 0x30e283190>,
 'x': array([6.34317545e-01, 2.30202606e+00, 3.70752286e-02, 1.58528377e-03,
       1.08352086e+00, 1.32461875e+00, 1.96341275e-01, 3.99999261e+00,
       3.99910324e+00, 3.17894999e+00, 1.62405979e+00, 4.00003652e+00,
       6.1651955

lsoda -- warning..internal t = 0.585178 and h_ = 4.60924e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.585178 and h_ = 4.60924e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.585178 and h_ = 3.58086e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.585178 and h_ = 3.58086e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.585178 and h_ = 2.81766e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.585178 and h_ = 2.81766e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsod

  100  75313 3.095218359412458e-02 6.3e+00 1.20e-01  5e-02  1e-01 0:04.7


...

  175 143734 5.111623538705691e-02 2.4e+01 1.43e-01  5e-02  2e-01 0:13.3
  200 144609 5.008865131324288e-02 3.6e+01 8.48e-02  3e-02  1e-01 0:15.4
  294 147899 1.489562605510054e-02 5.2e+01 2.30e-01  3e-02  3e-01 0:23.4
  300 148109 1.374516889118165e-02 5.2e+01 2.20e-01  3e-02  3e-01 0:24.0
  400 151609 1.083465665068395e-02 1.4e+02 1.75e-01  9e-03  3e-01 0:32.8
  500 155109 7.407270279751630e-03 2.0e+02 3.13e-01  1e-02  4e-01 0:41.5
  600 158609 4.783286012342910e-03 2.6e+02 2.97e-01  5e-03  3e-01 0:49.2
  700 162109 2.376965384351967e-03 5.0e+02 4.89e-01  4e-03  5e-01 0:57.2
  800 165609 1.970692937683837e-03 7.4e+02 3.11e-01  1e-03  2e-01 1:05.5
  900 169109 1.615107400020545e-03 8.7e+02 3.81e-01  1e-03  2e-01 1:13.6
 1000 172609 1.508611461516810e-03 1.8e+03 4.04e-01  9e-04  2e-01 1:21.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100 176109 1.497169889822445e-03 3.3e+03 1.04e-01  2e-04  5e-02 1:29.5
 1200 179609 1.495327283573017e-03 4.0e+03 2.73e-02 

lsoda -- warning..internal t = 0.237107 and h_ = 1.25825e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.237107 and h_ = 1.25825e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.237107 and h_ = 9.09982e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.237107 and h_ = 7.52786e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.237107 and h_ = 7.52786e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.237107 and h_ = 5.47521e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsod

   88 198669 3.887711174534356e-02 6.3e+00 2.77e-01  1e-01  3e-01 0:07.4
  100 199173 1.551036727719711e-02 7.8e+00 2.36e-01  8e-02  3e-01 0:08.5


lsoda -- warning..internal t = 0.512792 and h_ = 4.57338e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.512792 and h_ = 4.57338e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.512792 and h_ = 3.09691e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.512792 and h_ = 3.09691e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.512792 and h_ = 2.0971e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.512792 and h_ = 2.0971e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda 

  160 201693 6.057504514447209e-03 3.7e+01 1.32e-01  2e-02  2e-01 0:14.5
  200 203373 5.330815366206884e-03 1.0e+02 7.53e-02  1e-02  1e-01 0:18.5
  281 206775 4.427981963747973e-03 2.2e+02 2.56e-01  2e-02  4e-01 0:26.5
  300 207573 3.789978235216705e-03 2.5e+02 2.33e-01  1e-02  4e-01 0:28.4
  400 211773 2.027299319828641e-03 3.9e+02 1.10e-01  2e-03  1e-01 0:38.4
  500 215973 1.729704754093536e-03 5.5e+02 1.71e-01  3e-03  2e-01 0:48.3
  600 220173 1.616846238137265e-03 9.7e+02 6.55e-02  5e-04  8e-02 0:58.2
  700 224373 1.612606715365398e-03 1.5e+03 2.37e-02  7e-05  3e-02 1:08.1
  748 226389 1.611596129003628e-03 1.8e+03 7.53e-03  2e-05  8e-03 1:12.9
termination: {'tolfun': 1e-06}
{'compared': 31416,
 'evals': 225994,
 'evalsall': 226389,
 'f': 0.0016115914269646137,
 'last': <cma.utilities.utils.BlancClass object at 0x30ed32790>,
 'x': array([7.83194746e-01, 1.50398623e+00, 2.59590364e-02, 3.56782345e-04,
       1.10119602e+00, 1.18955332e-02, 1.19438298e+00, 4.00003863e+00,
       4.00

In [5]:
# find simplified models
loadpath = "example/init"
fname = "0000"

d, d_test = generate_d()
init_model = ChemKinetics(funcptr_cr, d, d_test=d_test)
init_model.load(loadpath, fname)
savepath = "example/sparse"
find_sparse_model(init_model, savepath)

save result: example/sparse/0000.csv
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 19 (seed=260836, Sun Apr 20 17:29:12 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 1.204073011208449e+00 1.0e+00 9.13e-01  9e-01  9e-01 0:00.0
    2     24 1.278494383934724e+00 1.1e+00 9.16e-01  9e-01  9e-01 0:00.1
    3     36 1.252931444026778e+00 1.2e+00 9.08e-01  9e-01  9e-01 0:00.1


lsoda -- warning..internal t = 0.0873472 and h_ = 6.62642e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.0873472 and h_ = 6.62642e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.0873472 and h_ = 4.81957e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.0873472 and h_ = 4.81957e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.0873472 and h_ = 3.48873e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.0873472 and h_ = 3.48873e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway

  100   1200 1.637330302681013e-03 3.4e+00 5.35e-03  2e-03  5e-03 0:02.6
  200   2400 1.559882582471862e-03 1.7e+01 2.85e-03  4e-04  4e-03 0:05.2
  300   3600 1.524423081518167e-03 5.7e+01 5.58e-03  5e-04  1e-02 0:07.8
  400   4800 1.497118201013739e-03 1.3e+02 2.53e-03  2e-04  5e-03 0:10.5
  500   6000 1.496868583985386e-03 1.5e+02 6.61e-04  5e-05  1e-03 0:13.1
  548   6576 1.496535255585284e-03 1.7e+02 3.77e-04  3e-05  8e-04 0:14.4
termination: {'tolfun': 1e-06}
{'compared': 6576,
 'evals': 5544,
 'evalsall': 6576,
 'f': 0.0014963868436004813,
 'last': <cma.utilities.utils.BlancClass object at 0x30ed17190>,
 'x': array([0.04044453, 1.6851078 , 0.01926515, 1.07268399, 0.46561007,
       0.45662239, 3.99664185, 4.00002244, 2.0160006 , 1.44755063,
       3.22254005, 3.50800509, 1.00005388, 3.30587556, 2.88692154,
       2.60274318, 1.42360577, 3.96481986, 1.92955476]),
 'x_geno': array([0.03993834, 1.6851078 , 0.01207278, 1.07268399, 0.46561007,
       0.45662239, 4.19172233, 4.25462639

lsoda -- warning..internal t = 0.212119 and h_ = 1.07365e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.212119 and h_ = 1.07365e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.212119 and h_ = 7.27031e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.212119 and h_ = 7.27031e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.212119 and h_ = 4.92314e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.212119 and h_ = 4.92314e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsod

   75  21358 1.978424127220446e-03 4.0e+00 2.31e-02  1e-02  2e-02 0:03.1
  100  21858 1.659817599925769e-03 6.9e+00 1.27e-02  3e-03  1e-02 0:04.2
  200  23858 1.504328325569070e-03 9.9e+01 7.67e-03  5e-04  2e-02 0:08.5
  300  25858 1.494103634087348e-03 1.6e+02 1.52e-03  8e-05  3e-03 0:12.9
  400  27858 1.493784670687430e-03 1.7e+02 5.47e-04  2e-05  1e-03 0:17.3
  409  28038 1.493593302188755e-03 1.7e+02 4.31e-04  2e-05  8e-04 0:17.7
termination: {'tolfun': 1e-06}
{'compared': 8180,
 'evals': 27487,
 'evalsall': 28038,
 'f': 0.0014933768950367537,
 'last': <cma.utilities.utils.BlancClass object at 0x30ef6c390>,
 'x': array([0.01928266, 1.68742254, 0.01986895, 1.07388154, 0.19900192,
       0.41643707, 3.99981945, 4.0000395 , 1.97496133, 1.42807124,
       3.20303779, 3.59007439, 1.0430053 , 3.26198076, 2.84051566,
       2.61725911, 1.41893462, 3.87837133, 1.92707802]),
 'x_geno': array([0.01210098, 1.68742254, 0.013038  , 1.07388154, 0.19900192,
       0.41643707, 4.23507981, 4.252026

...

  100   6808 1.234078820741511e-02 4.9e+01 4.29e-03  2e-04  3e-03 0:02.1
  145   7483 1.233796510434236e-02 2.1e+01 1.63e-03  4e-05  4e-04 0:03.1
termination: {'tolfun': 1e-06}
{'compared': 2175,
 'evals': 6711,
 'evalsall': 7483,
 'f': 0.012337524903731646,
 'last': <cma.utilities.utils.BlancClass object at 0x30efc7990>,
 'x': array([2.21360381, 0.11550587, 1.17539015, 3.97337251, 2.3785875 ,
       3.80638161]),
 'x_geno': array([2.21360381, 0.11550587, 1.17539015, 4.08673255, 2.3785875 ,
       3.80997344])}
save result: example/sparse/0025.csv
lam = 0.03072
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=239534, Sun Apr 20 18:00:10 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 7.771607059483804e-01 1.0e+00 9.86e-01  9e-01  1e+00 0:00.0
    2     18 4.337272194044861e-01 1.3e+00 9.46e-01  9e-01  1e+00 0:00.0
    3     27 4.324924461250131e-01 1.4e+00 8.79e-01  8e-01  9e-01 0:00.0


lsoda -- warning..internal t = 0.309913 and h_ = 1.98682e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.309913 and h_ = 1.98682e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.309913 and h_ = 1.34539e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.309913 and h_ = 1.34539e-17 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.309913 and h_ = 9.11037e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.309913 and h_ = 9.11037e-18 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsod

  100    900 3.226126454863925e-01 1.7e+01 4.88e-02  4e-03  3e-02 0:01.3
  200   1800 3.225435190692783e-01 1.7e+01 2.93e-04  9e-06  4e-05 0:02.6
  257   2313 3.225430989812347e-01 1.6e+01 1.55e-04  5e-06  1e-05 0:03.3
termination: {'tolfun': 1e-06}
{'compared': 2313,
 'evals': 2293,
 'evalsall': 2313,
 'f': 0.322543052682438,
 'last': <cma.utilities.utils.BlancClass object at 0x30efe9950>,
 'x': array([2.16019686, 0.44290404, 1.23438092, 3.44344853, 0.26028969,
       1.90246213]),
 'x_geno': array([2.16019686, 0.44290404, 1.23438092, 5.05664267, 0.26028969,
       1.90246213])}
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 6 (seed=313214, Sun Apr 20 18:00:13 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1   2303 2.233885555966341e+00 1.0e+00 9.19e-01  9e-01  1e+00 0:00.0
    2   2313 1.894959690053649e+00 1.2e+00 1.06e+00  9e-01  1e+00 0:00.0
    3   2323 7.131411715431495e-01 1.5e+00 1.18e+00  1e+00  1e+00 0:00.0


lsoda -- warning..internal t = 0.00741076 and h_ = 3.27163e-19 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.00741076 and h_ = 3.27163e-19 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.00741076 and h_ = 2.2154e-19 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.00741076 and h_ = 2.2154e-19 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0.00576947 and h_ = 3.86741e-19 are
         such that in the machine, t + h_ = t on the next step
lsoda -- warning..internal t = 0.00741076 and h_ = 1.50018e-19 are
         such that in the machine, t + h_ = t on the next step
         solver will continue anyway.
lsoda -- warning..internal t = 0

  100   3293 2.625672904758942e-01 1.4e+01 2.20e-02  2e-03  1e-02 0:01.3
  149   3783 2.625386361099707e-01 1.7e+01 1.48e-03  4e-05  3e-04 0:01.9
termination: {'tolfun': 1e-06}
{'compared': 1490,
 'evals': 3765,
 'evalsall': 3783,
 'f': 0.26253862016308144,
 'last': <cma.utilities.utils.BlancClass object at 0x30ee44550>,
 'x': array([2.25575288e+00, 1.76885427e-01, 1.05852416e+00, 7.73330925e-10,
       1.87524073e+00, 1.87819744e+00]),
 'x_geno': array([ 2.25575288, -0.27688543, -1.15852416, -0.05001244,  1.87524073,
       -1.97819744])}
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 6 (seed=317375, Sun Apr 20 18:00:15 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1   3777 1.074893301836341e+00 1.0e+00 1.02e+00  1e+00  1e+00 0:00.0
    2   3789 2.629216484015501e-01 1.2e+00 1.07e+00  1e+00  1e+00 0:00.0
    3   3801 2.629216484015501e-01 1.3e+00 9.87e-01  9e-01  1e+00 0:00.0
   26   4077 2.629216484015501e-01 3.1e+00 5.43e-02  2e-02  5e-02 0:00.4

In [6]:
# load a kinetic model and create a report
loadpath = "example/sparse"
fname = "0013"
d, d_test = generate_d()
ck = ChemKinetics(funcptr_cr, d, d_test=d_test)
ck.load(loadpath, fname)
print(chem_equation_with_oxalate(ck.l, ck.r, k=ck.k, chemformula=ck.chemformula, conservation=ck.conservation))
savename = "example/report.pdf"
report(loadpath, fname, savename)

( 1) Mn+7 + Mn+2     -> Mn+3 + Mn+6     k = 48.67
( 2) Mn+7 + C2O4-2   -> Mn+6 + C2O4-1   k = 0.03
( 3) Mn+3 + C2O4-2   -> Mn+2 + C2O4-1   k = 10.85
( 4) Mn+2 + Mn+6     -> 2 Mn+4          k = 9996.27
( 5) Mn+2 + Mn+4     -> 2 Mn+3          k = 9999.74
( 6) Mn+2 + C2O4-1   -> Mn+3 + C2O4-2   k = 54.30
( 7) C2O4-2 + Mn+4   -> Mn+3 + C2O4-1   k = 35.66
( 8) Mn+7 + C2O4-1   -> 2 CO2 + Mn+6    k = 1049.52
( 9) Mn+3 + C2O4-1   -> Mn+2 + 2 CO2    k = 385.73
(10) C2O4-2 + Mn+6   -> 2 CO2 + Mn+4    k = 478.75
(11) Mn+4 + C2O4-1   -> Mn+3 + 2 CO2    k = 9731.23

